# Lesson 5: Building NLP applications

### Student name / ID: Koki Yamashita /s1230041

---

First, set the data path and import some modules:

In [5]:
DATA_PATH = './'
import sys
sys.path.append (DATA_PATH)

In the last lesson we made some tools to prepare corpora for further processing. In this lesson, you'll implement two major applications that build upon the tools you developed. The first will be a relatively simple program that scores each text in a corpus according to its *Automatic Readability Index*. In the second application we will build a system that can predict who wrote a certain text. Again, we'll need to cover a lot of ground and things are becoming increasingly difficult now. So, let's get started!

## Automatic Readability Index

The *Automatic Readability Index* is a readability test designed to gauge the understandability of a text. The formula for calculating the *Automated Readability Index* is as follows:

$$ 4.71 \cdot \frac{nchars}{nwords} + 0.5 \cdot \frac{nwords}{nsents} - 21.43 $$

Let's apply some wishful thinking. If we had all the information needed to compute this formula, we could start with writing a function that does it for us. Let's do so.

#### Quiz!

Write a function `automatic_readability_index` that takes three arguments `n_chars`, `n_words` and `n_sents` and returns the ARI given those arguments.

In [6]:
def automatic_readability_index(n_chars, n_words, n_sents):
    # insert your code here
    return 4.71*(n_chars/n_words)+0.5*(n_words/n_sents)-21.43

# do not modify the code below, it is for testing your answer only!
# it should output True if you did well
print(abs(automatic_readability_index(300, 40, 10) - 15.895) < 0.001)

True


Now we need to write some code to obtain the numbers we so wishfully assumed to have. We will use the code we wrote in earlier chapters to read and tokenise texts. We stored all the functions we wrote for our corpus reader in `preprocess.py`. We only need the function `readcorpus` and import it here.

In [7]:
from pyhum.preprocess import readcorpus

Remember that the function readcorpus returns a generator of `(filename, sentences)` tuples. Sentences are represented by lists of strings, i.e. a list of tokens. Let's write a function `extract_counts` that takes a list of sentences as input and returns the number of characters, the number of words and the number of sentences as a tuple.

Let's write a function `extract_counts` that takes a list of sentences as input and returns the number of characters, the number of words and the number of sentences as a tuple.

#### Quiz!

Write the function `extract_counts`.

In [8]:
def extract_counts(sentences):
    # insert your code here
    n_sents=len(sentences)
    n_words=0
    n_chars=0
    for sent in sentences:
        n_words+=len(sent)
        for se in sent:
            n_chars+=len(se)
    
    return n_chars, n_words, n_sents

# do not modify the code below, for testing only!
print(extract_counts(
    [["This", "was", "rather", "easy", "."], 
     ["Please", "give", "me", "something", "more", "challenging"]]) == (54, 11, 2))

True


Well done! We're almost done. We could use our two functions to compute the ARI for a given text as follows:

In [9]:
sentences = [["This", "was", "rather", "easy", "."], 
             ["Please", "give", "me", "something", "more", "challenging"]]

n_chars, n_words, n_sents = extract_counts(sentences)

# do not modify the code below, it is for testing your answer only!
# it should output True if you did well
print(abs(automatic_readability_index(n_chars, n_words, n_sents) - 4.442) < 0.001)

True


However, it would be nice to have a little more abstraction.

#### Quiz!

Write the function `compute_ARI` that takes as argument a list of sentences (represented by lists of words) and returns the *Automatic Readability Index* for that input.

In [10]:
def compute_ARI(sentences):
    # insert your code here
    com = ()
    com=extract_counts(sentences)
    n_chars,n_words, n_sents=com
    return automatic_readability_index(n_chars,n_words, n_sents)
    
# do not modify the code below, it is for testing your answer only!
# it should output True if you did well
print(abs(compute_ARI(sentences) - 4.442) < 0.001)

True


Finally it would be nice to compare the readability of a number of texts. We need a function that iterates through the files in a directory and prints the Automatic Readability Index for each text. 

#### Quiz!

Write a function `compute_ARIs` that takes the name of a directory as input and prints the Automatic Readbility Index for each document in that directory.

In [11]:
from pyhum.preprocess import readcorpus, tokenise, split_sentences

def compute_ARIs(directory):
    # insert your code here
    for filename,text in readcorpus(directory):
        tokens=tokenise(text)
        senteces=split_sentences(tokens)
        print(filename + str(compute_ARI(sentences)))
    
compute_ARIs(DATA_PATH + 'data/')

austen-emma-excerpt-tokenised.txt4.441818181818181
austen-emma-excerpt.txt4.441818181818181
austen-emma.txt4.441818181818181
austen-frequency-distribution.txt4.441818181818181
twitter.txt4.441818181818181


----------

## Authorship attribution
Authorship attributionのアプリケーションのコアの実装

In this section you will implement the core of an authorship attribution application. You won't build a full stand-alone application, but rather focus on the core functions for classifying new texts for their authors.

The core of our application will be a naive bayes classifier. Following good programming principles, we will try to make this classifier as generic as possible. This allows us to use the classifier in other contexts than authorship attribution, such as text classification and classification in general.

The naive bayes classifier is a probabilistic classifier that, given a set of features, tries to find the class with the highest probability. It is based on applying Bayes' theorem and is called naive because of its strong independence assumption between features. This means that the absence or presence of each feature is assumed to be independent of each other. We compute the posterior probability of a class as the joint probability of all features given that class:

$$ P(y|x_1,\ldots,x_n) \propto P(y) \prod^n_{i=1} P(x_i|y)$$

Classification is based on the *maximum a posteriori* or MAP descision rule which simply picks the class (or author in our case) that is most probable:

$$ classify(x_1, \ldots, x_n) = \arg\max_y P(y) \prod^n_{i=1} P(x_i|y) $$

The main function we will implement has a simple job: take a text as an argument and classify it as being written by one of the authors. Let's again apply some wishful thinking and implement the function as follows:

In [12]:
def predict_author(text, feature_database):
    "Predict who wrote this text."
    return classify(score(extract_features(text), feature_database))

This function takes two arguments: the text to classify and the training data against which we want to classify the text. The function is basically an abstraction layer on top of `classify`,  `extract_features` and `score`. `classify` is a simple function that takes a dictionary of {$author_i$:  $P(author_i|text)$} and returns the author that is most probable. Let's implement this function.

#### Quiz!

Implement the function `classify`. It takes one argument, `scores` which is a dictionary with authors as keys and the probability of an author as value. Return the author with maximum probability. (Tip: use the built in function `max`, see [the documentation](http://docs.python.org/3/library/functions.html#max))

In [13]:
scores = {"Hermans": 0.15, "Voskuil": 0.55, "Reve": 0.2, "Mulisch": 0.18, "Claus": 0.02}

def classify(scores):
    # insert your code here
    return max(scores.keys())
    
print(classify(scores)== "Voskuil")

True


The function `extract_features` is rather straightforward as well. We'll build this function on top of the functions we defined in the previous lessons. For the moment we will assume that our model is a bag-of-words (BOW) model where the only features are individual words. We will define `extract_features` as an abstraction layer on top of `read_corpus_file` and `tokenise` as follows:

In [14]:
from pyhum.preprocess import read_corpus_file, tokenise

def extract_features(filename):
    return tokenise(read_corpus_file(filename))

Now for our training data, we need to store for each word how often it occurs with a particular author. As our data structure we will use a nested dictionary of the following structure `author -> word -> count`. We'll store the counts in the variable `feature_database`.

(For more information about `defaultdict`, see [the defaultdict documentation](http://docs.python.org/3/library/collections.html#collections.defaultdict).  For more information about `lambda` expressions, see [the lambda documentation](http://docs.python.org/3/tutorial/controlflow.html?highlight=lambda#lambda-expressions).)

In [15]:
from collections import defaultdict

feature_database = defaultdict(lambda: defaultdict(int))

To fill our `feature_database` we will need a couple of functions. First we need a function that returns the author of a particular text. To make things a little easier, we named our training files with the author's name followed by the title of the book, i.e. `austen-emma.txt`. Or when the path is part of the filename, `/path/to/austen-emma.txt`.

#### Quiz!

Write a function `extract_author` that takes a filename as input and returns the name of the author.

In [16]:
def extract_author(filename):
    # insert your code here
    return filename.split('/')[-1].split('-')[0]

# do not modify the code below, it is for testing your answer only!
# it should output True if you did well
print(extract_author("Austen-emma.txt") == "Austen")
print(extract_author("/path/to/Austen-emma.txt") == "Austen")

True
True


Next we'll need a function, `update_counts`, that takes as argument the name of an author and the words extracted using `extract_features` and adds these to our `feature_database`. The function should return a new updated version of the `feature_database`.

In [17]:
def update_counts(author, text, feature_database):
    # insert your code here
    for words in text:
        feature_database[author][words]+=1
        
    return feature_database

# do not modify the code below, for testing only!
feature_database = defaultdict(lambda: defaultdict(int))
feature_database = update_counts("Anonymous", "This was written with a lack of inspiration".split(), 
                                 feature_database)
test_database = defaultdict(lambda: defaultdict(int))
for word in "This was written with a lack of inspiration".split():
    test_database["Anonymous"][word] += 1
print(sorted(test_database.items())==sorted(feature_database.items()))

True


Finally we define a function `add_file_to_database` that takes a filename and the `feature_database` as input, extracts the author from the filename and adds the feature counts to the `feature_database`. We define it as follows:

In [18]:
def add_file_to_database(filename, feature_database):
    return update_counts(extract_author(filename), 
                         extract_features(filename), 
                         feature_database)

#### Quiz!

Now that we have a function to add one file to the `feature_database`, we need a function that adds an entire corpus to the database. Write a function that takes the name of a directory as input and add all files in this directory to the `feature_database`. Again, the function should return an updated version of our database.

In [19]:
import os

def add_directory_to_database(directory, feature_database):
    # insert your code here
    for filename in os.listdir(directory):
        feature_database = add_file_to_database(os.path.join(directory,filename),feature_database)
    return feature_database

We now have a function to extract the features of a document. We have also defined a couple of functions to add those features to our `feature_database`. It is now time to implement the core function of our authorship attribution application. 

Before we will implement the `score` function we will first implement a function to compute the probability of one feature given an author. There are two things to note with regard to this function.

First, if a given author and a word never occur together in the `feature_database`, the probabilty of that class will be zero (think about this, if you don't understand why). Needless to say this is rather problematic. A common strategy to surpass this problem is to add pseudocounts to the observed counts, normally 1. The pseudocounts need to be incorporated in both the numerator and the denominator.

Second, the probability of one feature will normally be quite small. If we now multiply all probabilities of our features given an author we will get a very small number, possibly too small, to be adequately represented by Python. Consider the code below:

In [20]:
x = 0.00000000000000001
for i in range(30):
    x = x * 0.000000000000001
    print(x)

1.0000000000000002e-32
1.0000000000000002e-47
1.0000000000000003e-62
1.0000000000000003e-77
1.0000000000000003e-92
1.0000000000000005e-107
1.0000000000000006e-122
1.0000000000000007e-137
1.0000000000000007e-152
1.0000000000000008e-167
1.0000000000000009e-182
1.000000000000001e-197
1.0000000000000011e-212
1.0000000000000012e-227
1.0000000000000012e-242
1.0000000000000013e-257
1.0000000000000014e-272
1.0000000000000015e-287
1.0000000000000015e-302
1e-317
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


After less than 30 multiplications, the values are too small for Python to distinguish them from each other. Even worse, the values default to zero and as we know multiplying by zero will return zero, and therefore all our probabilities could be zero! We therefore take the log of the individual feature probabilities and sum them to obtain our final score. Let's implement the `log_probability` function first.

In [21]:
from math import log

def log_probability(feature_counts, features_sum, n_features):
    return log((feature_counts + 1.0) / (features_sum + n_features))

`feature_counts` is the number of times the given feature occurs with a particular author. `features_sum` is the sum of all feature counts for that author. `n_features` is the number of unique features in our `feature_database` (needed for pseudocounts). Now that we have defined this crucial function, we are ready to implement our `score` function.

#### Quiz!

The function `score` takes as input a list of features and the `feature_database`. It should return a dictionary with authors as keys and their probabilities given the list of features as values. We'll provide the basic frame for this function below and ask you to fill in the details. This is without doubt the most challenging Quiz! you have seen so far and we will be very impressed if you get it right. 

In [27]:
def score(features, feature_database):
    "Predict who wrote the document on the basis of the corpus."
    scores = defaultdict(float)
    # compute the number of features in the feature database here
    n_f = len(set(feature for author, features in feature_database.items() for feature in features))
    print(author for author,features in feature_database.items())
    print(set(feature for author, features in feature_database.items() for feature in features))
    for author in feature_database:
        # compute the probability of features given that author here
        features_sum = sum(feature_database[author].values())
        for feature in features:
            scores[author] += log_probability(feature_database[author][feature],features_sum,n_f)
    return scores

# do not modify the code below, for testing your answer only! 
# It should return True if you did well!
features=["the", "a", "the", "be", "book"]
feature_database = defaultdict(lambda: defaultdict(int))
feature_database["A"]["the"] = 2
feature_database["A"]["a"] = 5
feature_database["A"]["book"]= 1
feature_database["B"]["the"] = 5
feature_database["B"]["a"] = 1
feature_database["B"]["book"] = 6
print(abs(dict(score(features, feature_database))["A"] - -7.30734) < 0.001)

<generator object score.<locals>.<genexpr> at 0x1116f3f10>
{'the', 'a', 'book'}
True


Wow! You have really done a great job. You have implemented almost entirely by yourself a complete naive bayes classifier that can be used for all kinds of classification problems, such as document classification and authorship attribution. 

Now we should put all things together and test how well our system works. In the folder `data/gutenberg/training` we have provided a couple of training documents from different author. The folder `data/gutenberg/testing` provides three test documents. Let's test our classifier on one of those documents!

In [64]:
# first define the feature_database
feature_database = defaultdict(lambda: defaultdict(int))
feature_database = add_directory_to_database(DATA_PATH + 'data/gutenberg/training', feature_database)
print(predict_author(DATA_PATH + 'data/gutenberg/testing/milton-poetical.txt', feature_database))

features =  ['Project', 'Gutenberg', "'", 's', 'The', 'Poetical', 'Works', 'of', 'John', 'Milton', ',', 'by', 'John', 'Milton', 'This', 'eBook', 'is', 'for', 'the', 'use', 'of', 'anyone', 'anywhere', 'at', 'no', 'cost', 'and', 'with', 'almost', 'no', 'restrictions', 'whatsoever', '.', 'You', 'may', 'copy', 'it', ',', 'give', 'it', 'away', 'or', 're-use', 'it', 'under', 'the', 'terms', 'of', 'the', 'Project', 'Gutenberg', 'License', 'included', 'with', 'this', 'eBook', 'or', 'online', 'at', 'www', '.', 'gutenberg', '.', 'org', 'Title', ':', 'The', 'Poetical', 'Works', 'of', 'John', 'Milton', 'Author', ':', 'John', 'Milton', 'Release', 'Date', ':', 'May', ',', '1999', '[', 'Etext', '#1745', ']', 'Posting', 'Date', ':', 'November', '20', ',', '2009', 'Language', ':', 'English', '***', 'START', 'OF', 'THIS', 'PROJECT', 'GUTENBERG', 'EBOOK', 'THE', 'POETICAL', 'WORKS', 'OF', 'JOHN', 'MILTON', '***', 'Produced', 'by', 'Donal', 'O', "'", 'Danachair', 'THE', 'POETICAL', 'WORKS', 'OF', 'JOHN', 

whitman


It would be nice to evaluate our classifier on more than one document and to obtain some sort of score of how well our classifier performs. We will implement two functions: `test_from_corpus` and `analyze_results`. 

#### Quiz!

`test_from_corpus` takes as input the name of a directory and a trained feature database. It then tries to predict the author of all files in the given directory. The function should return a list of `(ground-truth-author, predicted-author)` tuples.

In [65]:
def test_from_corpus(directory, feature_database):
    results = []
    # insert your code here
    for filename in os.listdir(directory):
        features = extract_features(os.path.join(directory,feature_database))
        author = extract_author(filename)
        predictuon = predict_author(features,feature_database)
        result.append(author,prediction)
    return results

Finally we will implement the function `analyze_results` which takes a list of `(ground-truth-author, predicted-author)` tuples as input and returns the accuracy of the classifier, which is defined as:

$$ accuracy(X) = \frac{\textrm{number of correct predictions}}{\textrm{total number of predictions}}$$

#### Quiz!

Implement the function `analyze_results` and test your classifier on the test corpus in `data/gutenberg/testing`:

In [66]:
def analyze_results(results):
    # insert your code here
    ans=0.0
    for author,predict in results:
        if author == predict:
            ans+=1
    return ans/len(results)

# do not modify the code below, for testing only!
print(analyze_results([("A", "A"), ("A", "B"), ("C", "C"), ("D", "C"), ("E", "E")]) == 0.6)

True


---------